In [ ]:
from aleph_alpha_client import TextControl
from dotenv import load_dotenv

from intelligence_layer.core import CompleteInput, Llama3InstructModel, NoOpTracer

load_dotenv()

# Attention Manipulation with `TextControl`




`TextControls` enable us to increase or decrease the attention of our model on different parts of the prompt (attention manipulation, AtMan).
This can be convenient for influencing the model's behavior and priorities or for understanding why a model generates a given completion.

First, we define the instruction and input of our model and run it without any AtMan.

In [ ]:
instruction_text = (
    "Say 'Hello' in one of the following languages. Say nothing else:\nLanguages: "
)
input_text = "Japanese and German"

llama_3_model = (
    Llama3InstructModel()
)  # `TextControl` is only supported for `InstructModel`
prompt_with_controls = llama_3_model.to_instruct_prompt(
    instruction=instruction_text,
    input=input_text,
)

complete_input = CompleteInput(prompt=prompt_with_controls)
output = llama_3_model.complete(complete_input, NoOpTracer())

print(output.completion)

As you can see, the model does not comply with the "one" part of our instruction and gives us both translations. Let's fix this behavior.

## Manipulating the Attention on the Instruction
To make the model only give us one translation, we increase the focus of the model on the word "one" . To this end, we create the `TextControl` for the instruction as follows: 

In [ ]:
highlight_of_one = "one"
instruct_controls = [
    TextControl(
        start=instruction_text.index(
            highlight_of_one
        ),  # Be careful to get the correct index
        length=len(highlight_of_one),
        factor=1.1,  # Increase focus on "one" by 10%
    )
]

prompt_with_controls = llama_3_model.to_instruct_prompt(
    instruction=instruction_text,
    input=input_text,
    instruction_controls=instruct_controls,
)

complete_input = CompleteInput(prompt=prompt_with_controls)
output = llama_3_model.complete(complete_input, NoOpTracer())

print(output.completion)

So this did not work. This is because we only increased the weight of the focus with the `factor` '1.1'. A `factor` of '1' would have no effect at all, and as it seems, an increase by 10% doesn't do the trick. So lets' increase it ten-fold. 

In [ ]:
instruct_controls += [
    TextControl(
        start=instruction_text.index(highlight_of_one),
        length=len(highlight_of_one),
        factor=10,
    )
]

prompt_with_controls = llama_3_model.to_instruct_prompt(
    instruction=instruction_text,
    input=input_text,
    instruction_controls=instruct_controls,
)

complete_input = CompleteInput(prompt=prompt_with_controls)
output = llama_3_model.complete(complete_input, NoOpTracer())

print(output.completion)

Finally, the model listens to the restriction. But what if we *also* want the model to be a bit less concise? 

### Using Multiple `TextControls`
We can apply multiple `TextControl`s for to different parts of our instruction. We can use this to only get one translation and a less concise answer: 

In [ ]:
highlight_of_conciseness = ". Say nothing else"
instruct_controls = [
    TextControl(
        start=instruction_text.index(highlight_of_one),
        length=len(highlight_of_one),
        factor=10,
    ),
    TextControl(
        start=instruction_text.index(highlight_of_conciseness),
        length=len(highlight_of_conciseness),
        factor=0.25,
    ),
]

prompt_with_controls = llama_3_model.to_instruct_prompt(
    instruction=instruction_text,
    input=input_text,
    instruction_controls=instruct_controls,
)

complete_input = CompleteInput(prompt=prompt_with_controls)
output = llama_3_model.complete(complete_input, NoOpTracer())

print(output.completion)

Feel free to experiment with the `factor` parameters of the `TextControl`s to see how the output changes. You will notice that are some sweet spots that change the output for the better or worse.

## Manipulating the Attention on the Input
We can also manipulate the attention on different parts of the input instead of the instruction. The procedure is the same, but we use the parameter `input_controls` of `to_instruct_prompt()`:

In [ ]:
instruct_controls = [
    TextControl(
        start=instruction_text.index(highlight_of_one),
        length=len(highlight_of_one),
        factor=10,
    ),
    TextControl(
        start=instruction_text.index(highlight_of_conciseness),
        length=len(highlight_of_conciseness),
        factor=0.3,  # Notice, how we need to tweak this up a bit to get only one answer
    ),
]

highlight_of_language = "German"
input_controls = [
    TextControl(
        start=input_text.index(highlight_of_language),
        length=len(highlight_of_language),
        factor=10,
    )
]

prompt_with_controls = llama_3_model.to_instruct_prompt(
    instruction=instruction_text,
    input=input_text,
    instruction_controls=instruct_controls,
    input_controls=input_controls,
)

complete_input = CompleteInput(prompt=prompt_with_controls)
output = llama_3_model.complete(complete_input, NoOpTracer())

print(output.completion)